In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS']='1'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from tf_keras.optimizers import Adam
from tf_keras.losses import SparseCategoricalCrossentropy
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

2025-12-23 16:04:59.643522: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 16:05:00.000898: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-23 16:05:01.846314: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWar

In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
tokenzier=AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [5]:
df=pd.read_csv('/home/chinghia/AI_Engineer/IMDB_Dataset/IMDB Dataset.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [7]:
X=df.drop('sentiment',axis=1)
y=df['sentiment']
X.head()

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."


In [8]:
def clean_code(text):
    text=re.sub(r'<.*?>',' ',text)
    text=re.sub(r'http\S+|www\S+|https\S+', ' ', text, flags=re.MULTILINE)
    text=re.sub(r'\s+',' ',text).strip()
    return text

In [9]:
X['review']=X['review'].apply(clean_code)
X.head(5)

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."


In [10]:
X=X.iloc[:,0]  #[x ,y:z ,e]
y=y.astype('category').cat.codes

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
def encode_data(text):
    encodings=tokenzier(
        text.tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='tf'
    )
    return encodings

In [ ]:
X_train=encode_data(X_train)
X_test=encode_data(X_test)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
modeltrans=TFAutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    use_safetensors=False)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
modeltrans.compile(
    optimizer=Adam(learning_rate=2e-5),#0.00002
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
modeltrans.fit(
    dict(X_train),y_train,
    epochs=1,
    batch_size=2,
    validation_data=(dict(X_test),y_test)
)

Epoch 1/2


2025-12-23 15:54:17.497922: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.32MiB (rounded to 7680000)requested by op tf_bert_for_sequence_classification/bert/encoder/layer_._5/attention/self/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-12-23 15:54:17.498376: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1049] BFCAllocator dump for GPU_0_bfc
2025-12-23 15:54:17.498427: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (256): 	Total Chunks: 65, Chunks in use: 64. 16.2KiB allocated for chunks. 16.0KiB in use in bin. 356B client-requested in use in bin.
2025-12-23 15:54:17.498431: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (512): 	Total Chunks: 1, Chunks in use: 0. 512B allocated for chunks. 0B in

ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification/bert/encoder/layer_._5/attention/self/Softmax defined at (most recent call last):
  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/chinghia/miniconda3/envs/ai_env/lib/python3.10/asyncio/base_events.py", line 603, in run_forever